<a href="https://colab.research.google.com/github/Madhukiranchiti/RTML-HW/blob/main/Assignment_2_Madhu_RTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2
## Course : Real-Time Machine Learning 5106
### Name : Madhu Kiran Chiti
### Student ID : 801333676
### Github Link :


In [2]:

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from matplotlib import pyplot as plt

import torch

from torch import nn
from torchvision import datasets,transforms

!pip install ptflops
import ptflops
from ptflops import get_model_complexity_info

from torch.utils.data import DataLoader, random_split, Subset

from torch.nn import functional as F
!pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy, BinaryAccuracy
from torchmetrics import ConfusionMatrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 11.0 MB/s eta 0:00:00


In [1]:
train_transform = transforms.Compose([
                                        transforms.ToTensor(),
                                      transforms.Resize(64),
                                        transforms.Normalize((0.29), (0.32)),
])

test_transform = transforms.Compose([
                                        transforms.ToTensor(),
                                     transforms.Resize(64),
                                        transforms.Normalize((0.29), (0.32)),
])

cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)


train_cifar, valid_cifar = random_split(cifar_trainset, [0.8, 0.2])
trainLoader = DataLoader(train_cifar, batch_size=256, shuffle=True, num_workers=1)
validateLoader = DataLoader(valid_cifar, batch_size=256, shuffle=True, num_workers=1)
testLoader = DataLoader(cifar_testset, batch_size=256, shuffle=True, num_workers=1)

NameError: name 'transforms' is not defined

In [2]:
class Train_Model: # A class to train and save the developed model and the metrics


    def __init__(self, model, loss, optimizer, accuracy, model_type, device, classes=0):

        self.device = device
        self.model = model.to(device) ## Setting the model on GPU
        self.Loss_Function = loss
        self.optimizer = optimizer
        self.accuracy = accuracy
        self.model_type = model_type
        self.classNum = classes

        # A dictionary to append or store the results
        self.Metrics = {"Training_Loss":[],"Training_Accuracy":[], "Validation_Loss":[], "Validation_Accuracy":[],"Test_Accuracy":0}
        self.ConfMatrix = None

    # Function to train the model
    def model_run(self, l_data,run_type):

        if run_type == 'train':
          self.model.train()

          if self.model_type == "Classification":
              MCA = self.accuracy(self.classNum)
          else:
              MCA = self.accuracy

          loss_sum = 0
          acc = 0

          # Iterates over the data
          for data, labels in tqdm(l_data):

              # transform data into one-hot vectors
              data = data.to(self.device)
              if self.model_type == "Classification":
                  labels = torch.eye(10)[labels]
              else:
                  labels = labels.reshape(-1, 1)
              labels = labels.to(self.device)

              predictions = self.model(data)
              loss_value = self.Loss_Function(predictions, labels)


              self.optimizer.zero_grad()
              loss_value.backward()
              self.optimizer.step()


              # Set the predictions and labels back into integers for accuracy calculation
              if self.model_type == "Classification":
                  predictions = torch.Tensor([torch.argmax(i).item() for i in predictions])
                  labels = torch.Tensor([torch.argmax(i).item() for i in labels])

              ##### Calculate Loss and accuracy
              loss_sum += loss_value.item()
              if self.model_type == "Classification":
                  acc += MCA(predictions, labels)

          self.Metrics["Training_Loss"].append(loss_sum / len(l_data))
          if self.model_type == "Classification":
              self.Metrics["Training_Accuracy"].append(acc / len(l_data))


        elif run_type == 'eval':
          self.model.eval()

          if self.model_type == "Classification":
              MCA = self.accuracy(self.classNum)
          else:
              MCA = self.accuracy

          loss_sum = 0
          acc = 0

          for data, labels in l_data:

              data = data.to(self.device)
              if self.model_type == "Classification":
                  labels = torch.eye(10)[labels]
              else:
                  labels = labels.reshape(-1, 1)
              labels = labels.to(self.device)

              with torch.no_grad():
                  predictions = self.model(data)
              loss_value = self.Loss_Function(predictions, labels)

              if self.model_type == "Classification":
                  predictions = torch.Tensor([torch.argmax(i).item() for i in predictions])
                  labels = torch.Tensor([torch.argmax(i).item() for i in labels])

              loss_sum += loss_value.item()
              if self.model_type == "Classification":
                  acc += MCA(predictions, labels)

          self.Metrics["Validation_Loss"].append(loss_sum / len(l_data))
          if self.model_type == "Classification":
              self.Metrics["Validation_Accuracy"].append(acc / len(l_data))


    def fit(self, t_data, v_data, EPOCHS):


        for i in range(EPOCHS):

            self.model_run(v_data,'eval')
            self.model_run(t_data,'train')

            print("Current Epoch:", i+1)
            print("Training_Loss:", self.Metrics["Training_Loss"][-1], " | Validation_Loss:", self.Metrics["Validation_Loss"][-1])
            if self.model_type == "Classification":
                print("Training_Accuracy:", self.Metrics["Training_Accuracy"][-1].item(), " | Validation_Accuracy:", self.Metrics["Validation_Accuracy"][-1].item())


    def Test_Model(self, testLoader):

        self.model.eval()

        if self.model_type == "Classification":
            confusion = ConfusionMatrix(task="multiclass", num_classes=self.classNum)
            MCA = self.accuracy(self.classNum)
        else:
            MCA = self.accuracy

        predMax = torch.empty(0)
        labelMax = torch.empty(0)

        for data, labels in testLoader:

            data = data.to(self.device)
            if self.model_type == "Classification":
                labels = torch.eye(10)[labels]
            else:
                labels = labels.reshape(-1, 1)
            labels = labels.to(self.device)

            with torch.no_grad():
                pred = self.model(data)

            if self.model_type == "Classification":
                pred = torch.Tensor([torch.argmax(i).item() for i in pred])
                labels = torch.Tensor([torch.argmax(i).item() for i in labels])

            predMax = torch.cat((predMax, pred))
            labelMax = torch.cat((labelMax, labels))

        if self.model_type == "Classification":
            self.ConfMatrix = confusion(predMax, labelMax)
            self.Metrics["Test_Accuracy"] = MCA(predMax, labelMax).item()




In [ ]:
def init_cnn(module):
  if type(module) == nn.Linear or type(module) == nn.Conv2d:
        nn.init.xavier_uniform_(module.weight)


In [ ]:
#@d2l.add_to_class(d2l.Trainer)
def __init__(self, max_epochs, num_gpus=1, gradient_clip_val=0):
        self.save_hyperparameters()
        self.train_loss = []
        self.train_acc = []
        self.valid_loss = []
        self.valid_acc = []

        assert num_gpus == 1,'No GPU support yet'


In [ ]:
# @d2l.add_to_class(d2l.Classifier)
def layer_summary(self, X_shape):
    X = torch.randn(*X_shape)
    for layer in self.net:
        X = layer(X)
        print(layer.__class__.__name__, 'output shape:\t', X.shape)



In [ ]:
#@d2l.add_to_class(d2l.Module)
def training_step(self, batch):
  l = self.loss(self(*batch[:-1]), batch[-1])
  self.plot('loss', l, train=True)
  self.train_loss.append(l)
  return l

#@d2l.add_to_class(d2l.Module)
def validation_step(self, batch):
  l = self.loss(self(*batch[:-1]), batch[-1])
  self.valid_loss.append(l)
  self.plot('loss', l, train=False)

In [ ]:
#@d2l.add_to_class(d2l.Module)
def plot(self, key, value, train):
        """Plot a point in animation."""
        assert hasattr(self, 'trainer'), 'Trainer is not inited'
        self.board.xlabel = 'epoch'
        if train:
            x = self.trainer.train_batch_idx / \
                self.trainer.num_train_batches
            n = self.trainer.num_train_batches / \
                self.plot_train_per_epoch
            metric_type = 'train'
        else:
            x = self.trainer.epoch + 1
            n = self.trainer.num_val_batches / \
                self.plot_valid_per_epoch
            metric_type = 'val'
        self.board.draw(x, d2l.numpy(d2l.to(value, d2l.cpu())),
                        ('train_' if train else 'val_') + key,
                        every_n=int(n))

        if not train and self.trainer.epoch % 1 == 0:
          train_loss = self.trainer.train_loss[-1]
          #train_acc = self.trainer.train_acc[-1]
          val_loss = self.trainer.valid_loss[-1]
          #val_acc = self.trainer.valid_acc[-1]
          print(f"Epoch [{self.trainer.epoch+1}/{self.trainer.epochs}]: "
                f"train_loss = {train_loss:.4f}, train_acc = {train_acc:.4f}, ")
               # f"val_loss = {val_loss:.4f}, val_acc = {val_acc:.4f}")

In [ ]:
#@d2l.add_to_class(d2l.Trainer)
def fit(self, model, data):
        self.prepare_data(data)
        self.prepare_model(model)
        self.optim = model.configure_optimizers()
        self.epoch = 0
        self.train_batch_idx = 0
        self.val_batch_idx = 0
        for self.epoch in range(self.max_epochs):
            self.fit_epoch()
            self.evaluate()

In [ ]:
device_to_use = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Problem 1:

In [ ]:
class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                        input_shape= input_shape,
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Flatten())
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(1000, activation= 'relu'))
        self.add(Dense(num_classes, activation= 'softmax'))

        self.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [ ]:
num_classes = 2
model = AlexNet((227, 227, 3), num_classes)

In [ ]:
EPOCHS = 100
BATCH_SIZE = 32
image_height = 227
image_width = 227
train_dir = "./content/train"
valid_dir = "./content/validation"
model_dir = "./my_model.h5"

In [ ]:
train_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=10,
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  shear_range=0.1,
                  zoom_range=0.1)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(image_height, image_width),
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode="categorical")

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(image_height, image_width),
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    seed=7,
                                                    shuffle=True,
                                                    class_mode="categorical"
                                                    )
train_num = train_generator.samples
valid_num = valid_generator.samples

In [ ]:
alexnet_arch = AlexNet(((2, 64), (2, 128), (2, 256), (2, 512)))

alexnet_Model = Train_Model(alexnet_arch, nn.CrossEntropyLoss(), torch.optim.SGD(alexnet_arch.parameters(), lr=0.1, momentum=0.9), MulticlassAccuracy, "Classification",  device_to_use, 10)

t0 = datetime.now()
alexnet_Model.fit(trainLoader, validateLoader, 10)
t1 = datetime.now()

alexnet_Model.Test_Model(testLoader)

macs, params = get_model_complexity_info(alexnet_Model.model, (3, 64, 64), as_strings=True, print_per_layer_stat=False, verbose=True)


print("Time Taken to Train:", t1 - t0)
print("alexnet_Model Train Accuracy:", alexnet_Model.Metrics["Training_Accuracy"][-1].item())
print("alexnet_Model Test Accuracy:", alexnet_Model.Metrics["Test_Accuracy"])


alexnet_Model.model.to("cpu")
print(f"Macs:{macs} | Params:{params}")


### PLOTTING METRICS
plt.plot(alexnet_Model.Metrics['Training_Accuracy'], label='Training Accuracy')
plt.plot(alexnet_Model.Metrics['Validation_Accuracy'], label='Validation Accuracy')
plt.plot(alexnet_Model.Metrics['Training_Loss'], label='Training Loss')
plt.plot(alexnet_Model.Metrics['Validation_Loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()

**Problem** 2:  
 ResNet-18

In [ ]:
class Residual(nn.Module):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, dropout_prob=0.0):
        super().__init__()
        self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1,
                                   stride=strides)
        self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
        else:
            self.conv3 = None
        self.dropout = nn.Dropout(dropout_prob)
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)


In [ ]:
class ResNet(nn.Module):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    def block(self, num_residuals, num_channels, first_block=False):
      blk = []
      for i in range(num_residuals):
          if i == 0 and not first_block:
              blk.append(Residual(num_channels, use_1x1conv=True, strides=2))
          else:
              blk.append(Residual(num_channels))
      return nn.Sequential(*blk)

    def __init__(self, arch, lr=0.1, num_classes=10, dropout_prob=0.0):
      super(ResNet, self).__init__()
      self.dropout_prob = dropout_prob
      self.net = nn.Sequential(self.b1())

      for i, b in enumerate(arch):
          self.net.add_module(f'b{i+2}', self.block(*b, first_block=(i==0)))
      self.net.add_module('last', nn.Sequential(
          nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
          nn.LazyLinear(num_classes)))
      self.net.apply(d2l.init_cnn)

    def forward(self, x):
        return self.net(x)




In [ ]:
resnet_18_arch = ResNet(((2, 64), (2, 128), (2, 256), (2, 512)), dropout_prob=0.5)

resnet_18_Model = Train_Model(resnet_18_arch, nn.CrossEntropyLoss(), torch.optim.SGD(resnet_18_arch.parameters(), lr=0.1, momentum=0.9), MulticlassAccuracy, "Classification",  device_to_use, 10)

t0 = datetime.now()
resnet_18_Model.fit(trainLoader, validateLoader, 10)
t1 = datetime.now()

resnet_18_Model.Test_Model(testLoader)

macs, params = get_model_complexity_info(resnet_18_Model.model, (3, 64, 64), as_strings=True, print_per_layer_stat=False, verbose=True)


print("Time Taken to Train:", t1 - t0)
print("ResNet-18 Model Train Accuracy:", resnet_18_Model.Metrics["Training_Accuracy"][-1].item())
print("ResNet-18 Model Test Accuracy:", resnet_18_Model.Metrics["Test_Accuracy"])


resnet_18_Model.model.to("cpu")
print(f"Macs:{macs} | Params:{params}")


### PLOTTING METRICS
plt.plot(resnet_18_Model.Metrics['Training_Accuracy'], label='Training Accuracy')
plt.plot(resnet_18_Model.Metrics['Validation_Accuracy'], label='Validation Accuracy')
plt.plot(resnet_18_Model.Metrics['Training_Loss'], label='Training Loss')
plt.plot(resnet_18_Model.Metrics['Validation_Loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()

In [ ]:
resnet_18_arch = ResNet(((2, 64), (2, 128), (2, 256), (2, 512)), dropout_prob=0.6)

resnet_18_Model = Train_Model(resnet_18_arch, nn.CrossEntropyLoss(), torch.optim.SGD(resnet_18_arch.parameters(), lr=0.1, momentum=0.9), MulticlassAccuracy, "Classification",  device_to_use, 10)

t0 = datetime.now()
resnet_18_Model.fit(trainLoader, validateLoader, 10)
t1 = datetime.now()

resnet_18_Model.Test_Model(testLoader)

macs, params = get_model_complexity_info(resnet_18_Model.model, (3, 64, 64), as_strings=True, print_per_layer_stat=False, verbose=True)


print("Time Taken to Train:", t1 - t0)
print("ResNet-18 Model Train Accuracy:", resnet_18_Model.Metrics["Training_Accuracy"][-1].item())
print("ResNet-18 Model Test Accuracy:", resnet_18_Model.Metrics["Test_Accuracy"])


resnet_18_Model.model.to("cpu")
print(f"Macs:{macs} | Params:{params}")


### PLOTTING METRICS
plt.plot(resnet_18_Model.Metrics['Training_Accuracy'], label='Training Accuracy')
plt.plot(resnet_18_Model.Metrics['Validation_Accuracy'], label='Validation Accuracy')
plt.plot(resnet_18_Model.Metrics['Training_Loss'], label='Training Loss')
plt.plot(resnet_18_Model.Metrics['Validation_Loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()

In [ ]:
resnet_18_arch = ResNet(((2, 64), (2, 128), (2, 256), (2, 512)), dropout_prob=0.7)

resnet_18_Model = Train_Model(resnet_18_arch, nn.CrossEntropyLoss(), torch.optim.SGD(resnet_18_arch.parameters(), lr=0.1, momentum=0.9), MulticlassAccuracy, "Classification",  device_to_use, 10)

t0 = datetime.now()
resnet_18_Model.fit(trainLoader, validateLoader, 10)
t1 = datetime.now()

resnet_18_Model.Test_Model(testLoader)

macs, params = get_model_complexity_info(resnet_18_Model.model, (3, 64, 64), as_strings=True, print_per_layer_stat=False, verbose=True)


print("Time Taken to Train:", t1 - t0)
print("ResNet-18 Model Train Accuracy:", resnet_18_Model.Metrics["Training_Accuracy"][-1].item())
print("ResNet-18 Model Test Accuracy:", resnet_18_Model.Metrics["Test_Accuracy"])


resnet_18_Model.model.to("cpu")
print(f"Macs:{macs} | Params:{params}")


### PLOTTING METRICS
plt.plot(resnet_18_Model.Metrics['Training_Accuracy'], label='Training Accuracy')
plt.plot(resnet_18_Model.Metrics['Validation_Accuracy'], label='Validation Accuracy')
plt.plot(resnet_18_Model.Metrics['Training_Loss'], label='Training Loss')
plt.plot(resnet_18_Model.Metrics['Validation_Loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()

Baseline ResNet-11 Model

In [ ]:
resnet_11_arch = ResNet(((1, 64), (2, 128), (2, 256), (2, 512)), dropout_prob=0.5)

resnet_11_Model = Train_Model(resnet_11_arch, nn.CrossEntropyLoss(), torch.optim.SGD(resnet_11_arch.parameters(), lr=0.1, momentum=0.9), MulticlassAccuracy, "Classification",  device_to_use, 10)

t0 = datetime.now()
resnet_11_Model.fit(trainLoader, validateLoader, 10)
t1 = datetime.now()

resnet_11_Model.Test_Model(testLoader)

macs, params = get_model_complexity_info(resnet_11_Model.model, (3, 64, 64), as_strings=True, print_per_layer_stat=False, verbose=True)


print("Time Taken to Train:", t1 - t0)
print("ResNet-11 Model Train Accuracy:", resnet_11_Model.Metrics["Training_Accuracy"][-1].item())
print("ResNet-11 Model Test Accuracy:", resnet_11_Model.Metrics["Test_Accuracy"])


resnet_11_Model.model.to("cpu")
print(f"Macs:{macs} | Params:{params}")


### PLOTTING METRICS
plt.plot(resnet_11_Model.Metrics['Training_Accuracy'], label='Training Accuracy')
plt.plot(resnet_11_Model.Metrics['Validation_Accuracy'], label='Validation Accuracy')
plt.plot(resnet_11_Model.Metrics['Training_Loss'], label='Training Loss')
plt.plot(resnet_11_Model.Metrics['Validation_Loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()

In [ ]:
resnet_11_arch = ResNet(((1, 64), (2, 128), (2, 256), (2, 512)), dropout_prob=0.6)

resnet_11_Model = Train_Model(resnet_11_arch, nn.CrossEntropyLoss(), torch.optim.SGD(resnet_11_arch.parameters(), lr=0.1, momentum=0.9), MulticlassAccuracy, "Classification",  device_to_use, 10)

t0 = datetime.now()
resnet_11_Model.fit(trainLoader, validateLoader, 10)
t1 = datetime.now()

resnet_11_Model.Test_Model(testLoader)

macs, params = get_model_complexity_info(resnet_11_Model.model, (3, 64, 64), as_strings=True, print_per_layer_stat=False, verbose=True)


print("Time Taken to Train:", t1 - t0)
print("ResNet-11 Model Train Accuracy:", resnet_11_Model.Metrics["Training_Accuracy"][-1].item())
print("ResNet-11 Model Test Accuracy:", resnet_11_Model.Metrics["Test_Accuracy"])


resnet_11_Model.model.to("cpu")
print(f"Macs:{macs} | Params:{params}")


### PLOTTING METRICS
plt.plot(resnet_11_Model.Metrics['Training_Accuracy'], label='Training Accuracy')
plt.plot(resnet_11_Model.Metrics['Validation_Accuracy'], label='Validation Accuracy')
plt.plot(resnet_11_Model.Metrics['Training_Loss'], label='Training Loss')
plt.plot(resnet_11_Model.Metrics['Validation_Loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()

In [ ]:
resnet_11_arch = ResNet(((1, 64), (2, 128), (2, 256), (2, 512)), dropout_prob=0.7)

resnet_11_Model = Train_Model(resnet_11_arch, nn.CrossEntropyLoss(), torch.optim.SGD(resnet_11_arch.parameters(), lr=0.1, momentum=0.9), MulticlassAccuracy, "Classification",  device_to_use, 10)

t0 = datetime.now()
resnet_11_Model.fit(trainLoader, validateLoader, 10)
t1 = datetime.now()

resnet_11_Model.Test_Model(testLoader)

macs, params = get_model_complexity_info(resnet_11_Model.model, (3, 64, 64), as_strings=True, print_per_layer_stat=False, verbose=True)


print("Time Taken to Train:", t1 - t0)
print("ResNet-11 Model Train Accuracy:", resnet_11_Model.Metrics["Training_Accuracy"][-1].item())
print("ResNet-11 Model Test Accuracy:", resnet_11_Model.Metrics["Test_Accuracy"])


resnet_11_Model.model.to("cpu")
print(f"Macs:{macs} | Params:{params}")


### PLOTTING METRICS
plt.plot(resnet_11_Model.Metrics['Training_Accuracy'], label='Training Accuracy')
plt.plot(resnet_11_Model.Metrics['Validation_Accuracy'], label='Validation Accuracy')
plt.plot(resnet_11_Model.Metrics['Training_Loss'], label='Training Loss')
plt.plot(resnet_11_Model.Metrics['Validation_Loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()